In [ ]:
# selection utils
def pick_xlsx_sheet(pxl: pd.ExcelFile):
    global sheet_select, column_select
    if column_select != None:
        column_select.close()
    if sheet_select != None:
        sheet_select.close()
    sheet_select = widgets.Dropdown(
        options=pxl.sheet_names,
        description='Sheet:',
        value=None
    )
    display(sheet_select)
    def callback(e):
        global sheet_select
        nonlocal pxl
        sheet_select.disabled = True
        parse_xlsx_sheet(pxl, sheet_select.value, parse_xlsx_sheet_interactive_posthook)
        sheet_select.disabled = False
    sheet_select.observe(callback, 'value')


def pick_sheet_column(psheet: pd.DataFrame):
    global column_select
    if column_select != None:
        column_select.close()
    if len(psheet.columns) == 1:
        use_sheet_column(psheet, psheet.columns[0], use_sheet_column_interactive_posthook)
        return
    column_select = widgets.Dropdown(
        options=psheet.columns,
        description='Column:',
        value=None
    )
    display(column_select)
    def callback(e):
        global column_select
        nonlocal psheet
        column_select.disabled = True
        use_sheet_column(psheet, column_select.value, use_sheet_column_interactive_posthook)
        column_select.disabled = False
    column_select.observe(callback, 'value')


# parsing utils
def parse_csv(buffer: bytes, posthook = None, *args, **kwargs):
    data = io.StringIO(buffer.decode('utf-8'))
    sheet_df = pd.read_csv(data)
    sheet_df = dataset_config["date_parser"](sheet_df)
    if posthook != None:
        return posthook(sheet_df, *args, **kwargs)


def parse_xlsx(buffer: bytes, posthook = None, *args, **kwargs):
    data = io.BytesIO(buffer)
    xl = pd.ExcelFile(data, engine='openpyxl')
    if posthook != None:
        return posthook(xl, *args, **kwargs)


def parse_xlsx_sheet(xl: pd.ExcelFile, sheet_name: str, posthook = None, *args, **kwargs):
    global dataset_config
    sheet_df = pd.read_excel(xl, sheet_name=sheet_name)
    sheet_df = dataset_config["date_parser"](sheet_df)
    if posthook != None:
        return posthook(sheet_df, sheet_name, *args, **kwargs)


def use_sheet_column(sheet: pd.DataFrame, column_name: str, posthook = None, *args, **kwargs):
    df = sheet[[column_name]]
    if posthook != None:
        return posthook(df, column_name, *args, **kwargs)


def parse_xlsx_sheet_interactive_posthook(psheet: pd.DataFrame, psheet_name: str) -> None:
    global sheet, sheet_name
    sheet = psheet
    sheet_name = psheet_name
    pick_sheet_column(sheet)


def parse_xlsx_interactive_posthook(pxl: pd.ExcelFile) -> None:
    global xl, parsing_value
    xl = pxl
    parsing_status.value = 'Reading complete'
    pick_xlsx_sheet(xl)


def parse_csv_interactive_posthook(psheet: pd.DataFrame) -> None:
    global sheet, parsing_status, sheet_select
    sheet = psheet
    parsing_status.value = 'Reading complete'
    if sheet_select != None:
        sheet_select.close()
    pick_sheet_column(sheet)


def use_sheet_column_interactive_posthook(column_df: pd.DataFrame, pcolumn_name: str) -> None:
    global df, column_name
    column_name = pcolumn_name
    df = column_df
    parsing_status.value = 'Parsing complete'


def parse_uploaded_file_interactive(filename: str, buffer: bytes):
    ext = os.path.splitext(filename)[1][1:]
    if ext == 'csv':
        return parse_csv(buffer, posthook=parse_csv_interactive_posthook)
    elif ext == 'xlsx':
        return parse_xlsx(buffer, posthook=parse_xlsx_interactive_posthook)
    else:
        raise Exception(f'Unsupported file extension: {ext}')


def parse_xlsx_sync_posthook(xl_file: pd.ExcelFile, sheet_name: str, column_name: str) -> None:
    return parse_xlsx_sheet(xl_file, sheet_name, parse_xlsx_sheet_sync_posthook, column_name)


def parse_xlsx_sheet_sync_posthook(sheet_df: pd.DataFrame, sheet_name: str, column_name: str) -> None:
    return use_sheet_column(sheet_df, column_name, lambda df, _: df)


def parse_csv_sync_posthook(sheet_df: pd.DataFrame, column_name: str) -> None:
    return use_sheet_column(sheet_df, column_name, lambda df, _: df)


def parse_uploaded_file_sync(filename: str, buffer: bytes, sheet_name: str = None, column_name: str = None) -> pd.DataFrame:
    ext = os.path.splitext(filename)[1][1:]
    if ext == 'csv':
        return parse_csv(buffer, posthook=parse_csv_sync_posthook, sheet_name=sheet_name, column_name=column_name)
    elif ext == 'xlsx':
        return parse_xlsx(buffer, posthook=parse_xlsx_sync_posthook, sheet_name=sheet_name, column_name=column_name)
    else:
        raise Exception(f'Unsupported file extension: {ext}')
